<a href="https://colab.research.google.com/github/mxgdxbz/agent/blob/main/RAG_Cobenfy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai scikit-learn numpy sentence-transformers PyMuPDF langchain ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
from google.colab import userdata

from openai import OpenAI
import fitz
from sentence_transformers import SentenceTransformer
import openai
import sqlite3
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.text_splitter import RecursiveCharacterTextSplitter # import the missing library


In [3]:
# Set up OpenAI API
openai.api_key=userdata.get('OPENAI_API_KEY')

# Load multilingual embedding model (E5-small model)
model = SentenceTransformer('intfloat/multilingual-e5-small')

# Database setup (SQLite for simplicity)
conn = sqlite3.connect('embeddings.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS embeddings
              (chunk_id INTEGER PRIMARY KEY, text_chunk TEXT, embedding BLOB)''') #BLOB (Binary Large Object)
conn.commit()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [6]:

# Step1 - Function to read PDF and split into chunks
def read_pdf(file_path, chunk_size=500):
    doc = fitz.open(file_path)
    concatenated_text = ""
    for page in doc:
      text=page.get_text()
      concatenated_text+=text

    # Split text into chunks
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200) #play with chunk_size, no context lost
    chunks=text_splitter.split_text(concatenated_text)

    return chunks

# Step2 - Function to embed chunks and store in the database
def embed_and_store_chunks(chunks):
    for i, chunk in enumerate(chunks):
        embedding = model.encode(chunk)
        # Store embedding and chunk in the database (BLOB format for embedding)
        cursor.execute("INSERT INTO embeddings (text_chunk, embedding) VALUES (?, ?)",
                       (chunk, embedding.tobytes()))
    conn.commit()

from sklearn.preprocessing import normalize

# Step3 - Calculate similarity and retrieve top 5 chunks
def retrieve_relevant_chunks(query, top_n=5):
    # Encode the query
    query_embedding = model.encode(query)

    # Normalize the query embedding
    query_embedding = normalize([query_embedding])[0]

    # Retrieve all embeddings from the database
    cursor.execute("SELECT text_chunk, embedding FROM embeddings")
    all_embeddings = cursor.fetchall()

    similarities = []
    for text_chunk, embedding in all_embeddings:
        embedding_np = np.frombuffer(embedding, dtype=np.float32)
        # Normalize the chunk embedding
        embedding_np = normalize([embedding_np])[0]
        similarity = cosine_similarity([query_embedding], [embedding_np])[0][0]
        similarities.append((similarity, text_chunk))

    similarities.sort(reverse=True, key=lambda x: x[0])
    return [chunk for _, chunk in similarities[:top_n]]

# Function to generate a response using GPT-4 with the retrieved context
def generate_gpt4_response(query, relevant_chunks):
    # Combine the relevant chunks to form the context
    client = OpenAI(api_key=openai.api_key)
    context = "\n".join(relevant_chunks)
    prompt = f"Based on the following context:\n\n{context}\n\nAnswer the query: {query}"

    # Call GPT-4o with the context
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
  ],
        max_tokens=500,
        temperature=0.7
    )

    return response.choices[0].message.content



In [11]:
# Example usage
if __name__ == "__main__":
    # Step 1: Read and process the PDF file
    pdf_file_path = '/content/Cobenfy_Drug.com.pdf'  # Replace with your PDF path
    pdf_chunks= read_pdf(pdf_file_path)

    # Step 2: Embed chunks and store them in the database
    embed_and_store_chunks(pdf_chunks)

    # Step 3: Query the system and retrieve relevant chunks
    #user_query = "Could you provide a short summary of this new drug?"  # Replace with your query
    user_query = "Based on provided documents, what's the recommended dosage for patients just started with Cobenfy?"
    #user_query = "Based on provided documents, what's the active ingredients of Cobenfy?"
    relevant_chunks = retrieve_relevant_chunks(user_query)
    #print(relevant_chunks)

    # Step 4: Use GPT-4 to generate an augmented response using the relevant chunks
    gpt4_response = generate_gpt4_response(user_query, relevant_chunks)

    # Output the final response
    print("GPT-4 Response:", gpt4_response)

GPT-4 Response: The recommended starting dosage for patients just starting with Cobenfy is 50mg/20mg orally twice daily for at least two days. After that, the dosage can be increased to 100mg/20mg twice daily for at least five days. Depending on patient tolerability, the dosage may then be increased to 125mg/30mg orally twice daily.


In [8]:
client = OpenAI(api_key=openai.api_key)
response_raw = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_query}
  ],
        max_tokens=500,
        temperature=0.7
    )
response_raw

ChatCompletion(id='chatcmpl-BLaPrnSdKe1nyZRTCSpNjHVIAK04V', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I’m sorry, but I don’t have access to specific documents or databases that would provide the active ingredients of a product called Cobenfy. However, if Cobenfy is a medication or supplement, you can usually find its active ingredients listed on the packaging, in the prescribing information, or on the manufacturer's website. If you have access to any specific information or documents, feel free to share details, and I'll do my best to help!", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744484571, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_44added55e', usage=CompletionUsage(completion_tokens=90, prompt_tokens=31, total_tokens=121, completion_tokens_details=CompletionTokensDetails(accepted_prediction_toke

In [9]:
print( pdf_chunks)

['Cobenfy\nPronunciation: co-BEN-fee\nGeneric name: xanomeline and trospium chloride\nDosage form: capsules (50mg/20mg, 100mg/20mg, 125mg/30mg) xanomeline/trospium\nWhat is Cobenfy?\nCobenfy (xanomeline and trospium chloride) is a new treatment for schizophrenia used to improve\npositive and negative symptoms. Cobenfy contains xanomeline, which helps schizophrenia symptoms,\nand trospium, which works to reduce the side effects. Cobenfy is a new class of medicine and works', 'positive and negative symptoms. Cobenfy contains xanomeline, which helps schizophrenia symptoms,\nand trospium, which works to reduce the side effects. Cobenfy is a new class of medicine and works\ndifferently than traditional schizophrenia medications so it does not have a boxed warning and atypical\nantipsychotic class warnings and precautions.\nSchizophrenia is related to imbalances in messenger systems in the brain involving muscarinic', 'antipsychotic class warnings and precautions.\nSchizophrenia is related t

In [10]:
query_embedding = model.encode( "Based on provided documents, what's the active ingredients of Cobenfy?")
query_embedding

array([ 2.33055670e-02, -1.27098300e-02, -6.18806444e-02, -8.31315070e-02,
        8.96009356e-02, -4.34791595e-02,  6.61762208e-02,  4.26982529e-02,
        6.86715469e-02,  1.10365562e-02,  6.64764866e-02,  5.98796830e-03,
        7.06419945e-02, -7.42160305e-02, -5.25279157e-03,  2.86570601e-02,
        9.94242951e-02, -3.31455991e-02, -4.69452627e-02, -9.58886370e-02,
        2.70492572e-04, -1.50317396e-03, -2.20550951e-02,  2.89488435e-02,
        4.89669740e-02,  6.39613569e-02, -1.73101723e-02, -5.72340749e-02,
        3.96905839e-02, -6.40477911e-02, -5.66355996e-02, -6.51604170e-03,
        3.26958522e-02, -5.96142970e-02,  2.96401675e-03,  5.72561659e-02,
       -9.30435658e-02, -5.39350100e-02,  7.97213018e-02, -5.55343553e-02,
       -5.42671755e-02, -4.12242254e-03,  4.97537293e-02,  6.20121993e-02,
        8.64139348e-02,  6.83488473e-02, -2.01818757e-02,  4.51711081e-02,
       -1.97056271e-02, -3.89088988e-02, -1.75096784e-02,  5.25902025e-02,
       -1.17256586e-02,  

In [12]:
# More Questions
relevant_chunks

['titration. The maximum recommended dosage is 100mg/20mg twice daily. (2.3)\nGeneral dosing information\nTake at least 1 hour before a meal or at least 2 hours after a meal.\xa0\nDo not open capsules. (2.2)\xa0\nAssess liver enzymes and bilirubin before starting treatment with this medicine and as clinically\nindicated during treatment.\xa0\nAssess heart rate at baseline and as clinically indicated during treatment with Cobenfy.\nCobenfy capsules are available in strengths: 50mg/20mg, 100mg/20mg, 125mg/30mg',
 'titration. The maximum recommended dosage is 100mg/20mg twice daily. (2.3)\nGeneral dosing information\nTake at least 1 hour before a meal or at least 2 hours after a meal.\xa0\nDo not open capsules. (2.2)\xa0\nAssess liver enzymes and bilirubin before starting treatment with this medicine and as clinically\nindicated during treatment.\xa0\nAssess heart rate at baseline and as clinically indicated during treatment with Cobenfy.\nCobenfy capsules are available in strengths: 50mg

## RAGas

Evaluation of Rag Metric

In [14]:
# Make sure four components are matched
from datasets import Dataset
questions = ["What are the active ingredients in Cobenfy?"]
ground_truths = ["Cobenfy contains xanomeline and trospium chloride. Xanomeline acts on brain receptors, while trospium reduces peripheral side effects."]
answers = [gpt4_response]
contexts = [relevant_chunks]
# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "reference": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)


In [15]:
import os
from ragas import evaluate
from google.colab import userdata
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

# Get your API key from Colab secret manager
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
 # Manage the API key using the context manager
result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)


df = result.to_pandas()
df

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,What are the active ingredients in Cobenfy?,[titration. The maximum recommended dosage is ...,The recommended starting dosage for patients j...,Cobenfy contains xanomeline and trospium chlor...,0.2,0.5,1.0,0.880881


In [ ]:
# Try three question sets
questions = [
    "What are the active ingredients in Cobenfy?",
    "How should Cobenfy be taken?",
    "Who should not take Cobenfy?",
]

ground_truths = [
    "Cobenfy contains xanomeline and trospium chloride. Xanomeline acts on brain receptors, while trospium reduces peripheral side effects.",
    "It should be taken twice daily on an empty stomach. Capsules must not be opened.",
    "People with urinary retention, moderate/severe liver problems, or untreated narrow-angle glaucoma should avoid it. It’s also contraindicated in those allergic to trospium.",
]

# Step 3: Loop through questions and get answers + contexts
answers = []
contexts = []

for question in questions:
    relevant_chunks = retrieve_relevant_chunks(question)
    gpt4_response = generate_gpt4_response(question, relevant_chunks)

    answers.append(gpt4_response)
    contexts.append(relevant_chunks)

# Step 4: Create dataset
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "reference": ground_truths,
}

dataset = Dataset.from_dict(data)

# Step 5: Evaluate
result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

# Step 6: View results
df = result.to_pandas()
df

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,What are the active ingredients in Cobenfy?,[Cobenfy ingredients \nActive ingredients: xa...,The active ingredients in Cobenfy are xanomeli...,Cobenfy contains xanomeline and trospium chlor...,1.0,1.0,1.00,1.0
1,How should Cobenfy be taken?,[about the health of women exposed to Cobenfy ...,Cobenfy should be taken as follows:\n\n- Take ...,It should be taken twice daily on an empty sto...,1.0,0.5,0.75,1.0
2,Who should not take Cobenfy?,[Who should not take this medicine?\nCobenfy s...,Cobenfy should not be taken by individuals who...,"People with urinary retention, moderate/severe...",1.0,1.0,1.00,1.0
